In [2]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import mysql.connector
import ssl

# konektor ke database
db = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='UnNgGrape'
        )
cursor = db.cursor()
exist = 0
add = 0

# scrape berdasarkan data keyword
cursor.execute("SELECT * FROM keyword")
raw_keyword = cursor.fetchall()
for row2 in raw_keyword:
    # penyiapan scraping
    keyword = row2[0]
    
    try:
        r = Request("https://www.jobstreet.co.id/id/job-search/" + keyword +"-jobs/", headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
        gcontext = ssl.SSLContext()  # Only for gangstars
        response = urlopen(r, context=gcontext).read()
        soup = BeautifulSoup(response, "lxml")
    except:
        continue

    # inisialisasi array untuk menampilkan hasil
    title_lowongan = []
    nama_perusahaan = []
    lokasi_perusahaan = []
    link_lowongan = []

    jobList = soup.find_all("div","sx2jih0 zcydq82a _27u74_0")
    for p in jobList:
        link = "https://www.jobstreet.co.id"+p.find('a').get("href")
        
        # mengecek apakah data sudah adat atau belum
        cursor.execute("SELECT * FROM lowongan WHERE link_lowongan = '" + link + "'")
        data = cursor.fetchall()
        # if else untuk pengecekan input data
        if data:
            exist= exist + 1
        else:
            asal = "jobstreet"

            lowongan = p.find("h1", "sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc3 _18qlyvc8").get_text()
            # menggunakan try except karena ada beberapa perusahaan yang dirahasiakan
            try:
                perusahaan = p.find("span", "sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc1 _18qlyvc8").get_text()
            except:
                perusahaan = "Perusahaan Dirahasiakan"
            lokasi = p.find("span", "sx2jih0 zcydq82q zcydq810 iwjz4h0").get_text()
        
            cursor.execute(
                        "INSERT INTO lowongan(asal, title_lowongan, nama_perusahaan, lokasi_perusahaan, link_lowongan)"
                        "VALUES ('"+ asal +"', '"+ lowongan.replace(',', '').replace("'", "").replace('"', '') +"', '"+ perusahaan.replace(',', '').replace("'", "").replace('"', '') +"', '"+ lokasi.replace(',', '').replace("'", "").replace('"', '') +"', '"+ link +"')"
                        )
            
            db.commit()
            add = add + 1
        
            title_lowongan.append(lowongan)
            nama_perusahaan.append(perusahaan)
            lokasi_perusahaan.append(lokasi)
            link_lowongan.append(link)

print("Data exists: " + str(exist))
print("Data has been added: " + str(add))
jobList_dict ={'lowongan':title_lowongan, 'perusahaan':nama_perusahaan, 'lokasi':lokasi_perusahaan, 'link':link_lowongan}
df = pd.DataFrame(jobList_dict,columns = ['lowongan','perusahaan', 'lokasi', 'link'])

df.sort_values('lowongan',ascending=True)

Data exists: 0
Data has been added: 0


,lowongan,perusahaan,lokasi,link
6,Accounting Section Head,PT Sri Trang Lingga Indonesia,Palembang,https://www.jobstreet.co.id/id/job/accounting-...
2,Administrasi Support (Indonesia Barat),Perusahaan Dirahasiakan,Perusahaan Dirahasiakan,https://www.jobstreet.co.id/id/job/administras...
4,Area Sales Manager ( Sumatera Selatan ),PT URC Indonesia,Palembang,https://www.jobstreet.co.id/id/job/area-sales-...
1,Branch Manager,PT Delisari Nusantara,"Banjarmasin, Malang, Makassar, Palembang, Peka...",https://www.jobstreet.co.id/id/job/branch-mana...
3,Business Representative Area Palembang,PT Setia Anugrah Medika,"Palembang, Lampung",https://www.jobstreet.co.id/id/job/business-re...
0,Manager Underwriting,PT Asuransi Umum Bumiputera Muda 1967,"Palembang, Bangka Belitung, Manado, Pekanbaru,...",https://www.jobstreet.co.id/id/job/manager-und...
5,SPG / SPB,PT Propan Raya ICC,"Palembang, Lampung, Pekanbaru, Medan",https://www.jobstreet.co.id/id/job/spg-spb-369...
7,Sales Representative (Medical),PT. Budimanmaju Megah Farmasi,"Jakarta Raya, Bogor, Bandung, Batam, Medan, Pa...",https://www.jobstreet.co.id/id/job/sales-repre...
